In [1]:
#from keras.applications import InceptionV3
from keras.applications import VGG16

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#conv_base = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

In [3]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 299, 299, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 299, 299, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 299, 299, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 149, 149, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 149, 149, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 149, 149, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 74, 74, 128)       0         
__________

In [4]:
from keras import models
from keras import layers
from keras import regularizers

In [5]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.normalization.BatchNormalization())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
# model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 9, 9, 512)         14714688  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 512)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 1285      
Total params: 14,849,349
Trainable params: 14,848,325
Non-trainable params: 1,024
____________________________________________________________

In [6]:
print(len(model.trainable_weights))
conv_base.trainable = False
print(len(model.trainable_weights))

32
6


In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 9, 9, 512)         14714688  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 512)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 1285      
Total params: 14,849,349
Trainable params: 133,637
Non-trainable params: 14,715,712
__________________________________________________________

In [8]:
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

In [9]:
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, 
                                   shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)

In [10]:
train_dir = r'E:\6--datasets\flower\train'
test_dir = r'E:\6--datasets\flower\test'
validation_dir = r'E:\6--datasets\flower\validation'

In [11]:
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(299, 299), batch_size=32, class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(validation_dir, target_size=(299, 299), batch_size=32, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(299, 299), batch_size=32, class_mode='categorical')

Found 2942 images belonging to 5 classes.
Found 364 images belonging to 5 classes.
Found 364 images belonging to 5 classes.


In [12]:
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(1e-4), metrics=['acc'])

In [13]:
import time
def print_current():
    print (time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print_current()

2019-04-25 11:28:03


In [14]:
print_current()
model.fit_generator(train_generator, steps_per_epoch=92, epochs=30, validation_data=validation_generator, validation_steps=12)
print_current()

2019-04-25 11:28:03
Epoch 1/30
92/92 [==============================] - 63s 689ms/step - loss: 1.5366 - acc: 0.3981 - val_loss: 1.0993 - val_acc: 0.5989
Epoch 2/30
92/92 [==============================] - 53s 581ms/step - loss: 1.1209 - acc: 0.5623 - val_loss: 0.8884 - val_acc: 0.6401
Epoch 3/30
92/92 [==============================] - 56s 606ms/step - loss: 0.9285 - acc: 0.6410 - val_loss: 0.7823 - val_acc: 0.6978
Epoch 4/30
92/92 [==============================] - 55s 603ms/step - loss: 0.8749 - acc: 0.6614 - val_loss: 0.7246 - val_acc: 0.7280
Epoch 5/30
92/92 [==============================] - 55s 593ms/step - loss: 0.7818 - acc: 0.6978 - val_loss: 0.6782 - val_acc: 0.7665
Epoch 6/30
92/92 [==============================] - 55s 593ms/step - loss: 0.7511 - acc: 0.7240 - val_loss: 0.6458 - val_acc: 0.7692
Epoch 7/30
92/92 [==============================] - 55s 594ms/step - loss: 0.7165 - acc: 0.7293 - val_loss: 0.6259 - val_acc: 0.7802
Epoch 8/30
92/92 [==============================]

In [15]:
test_loss, test_acc = model.evaluate_generator(test_generator, steps=12)
print(test_loss, test_acc)

0.49616377301268527 0.8461538461538461


In [ ]:
print_current()
model.fit_generator(train_generator, steps_per_epoch=92, epochs=100, validation_data=validation_generator, validation_steps=12)
print_current()

2019-04-25 12:04:31
Epoch 1/100
92/92 [==============================] - 59s 640ms/step - loss: 0.4278 - acc: 0.8375 - val_loss: 0.4678 - val_acc: 0.8571
Epoch 2/100
92/92 [==============================] - 55s 594ms/step - loss: 0.4285 - acc: 0.8447 - val_loss: 0.4826 - val_acc: 0.8462
Epoch 3/100
92/92 [==============================] - 55s 595ms/step - loss: 0.4228 - acc: 0.8453 - val_loss: 0.5191 - val_acc: 0.8352
Epoch 4/100
92/92 [==============================] - 57s 616ms/step - loss: 0.4456 - acc: 0.8348 - val_loss: 0.4797 - val_acc: 0.8379
Epoch 5/100
92/92 [==============================] - 55s 595ms/step - loss: 0.4100 - acc: 0.8494 - val_loss: 0.4743 - val_acc: 0.8489
Epoch 6/100
92/92 [==============================] - 55s 596ms/step - loss: 0.4021 - acc: 0.8556 - val_loss: 0.4715 - val_acc: 0.8462
Epoch 7/100
92/92 [==============================] - 56s 608ms/step - loss: 0.3981 - acc: 0.8552 - val_loss: 0.4689 - val_acc: 0.8434
Epoch 8/100
92/92 [=======================

92/92 [==============================] - 56s 605ms/step - loss: 0.2843 - acc: 0.9008 - val_loss: 0.4229 - val_acc: 0.8654
Epoch 62/100
92/92 [==============================] - 55s 599ms/step - loss: 0.2871 - acc: 0.8933 - val_loss: 0.4760 - val_acc: 0.8599
Epoch 63/100
92/92 [==============================] - 56s 612ms/step - loss: 0.3004 - acc: 0.8861 - val_loss: 0.4701 - val_acc: 0.8599
Epoch 64/100
92/92 [==============================] - 56s 610ms/step - loss: 0.2731 - acc: 0.9041 - val_loss: 0.4780 - val_acc: 0.8571
Epoch 65/100
92/92 [==============================] - 56s 604ms/step - loss: 0.2889 - acc: 0.8881 - val_loss: 0.4546 - val_acc: 0.8626
Epoch 66/100
92/92 [==============================] - 56s 607ms/step - loss: 0.2601 - acc: 0.9073 - val_loss: 0.4596 - val_acc: 0.8654
Epoch 67/100
92/92 [==============================] - 56s 612ms/step - loss: 0.2734 - acc: 0.9011 - val_loss: 0.4807 - val_acc: 0.8544
Epoch 68/100
92/92 [==============================] - 56s 608ms/step

In [ ]:
test_loss, test_acc = model.evaluate_generator(test_generator, steps=12)
print(test_loss, test_acc)